# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:

            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create Tables and Queries

### Query 1: 
**Request**: Select the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

The following query achieves the desired results:

```sql
SELECT artist, title, length 
FROM songs_in_session 
WHERE sessionId = 338 AND itemInSession = 4
```

* The table name is be `songs_in_session`.
* Table attributes are: `sessionId`, `itemInSession`, `artist`, `title` and `length`.
* The primary key consists of a partition key `sessionId`, and clustering key `itemInSession` for the purpose of filtering.


#### Create Table

In [8]:
query = """
CREATE TABLE IF NOT EXISTS songs_in_session (
    sessionId INT, 
    itemInSession INT, 
    artist TEXT, 
    title TEXT, 
    length FLOAT, 
    PRIMARY KEY (sessionId, itemInSession)
    )
"""

try:
    session.execute(query)
except Exception as e:
    print(e)  

#### Insert Data

In [9]:
file = 'event_datafile_new.csv'
query = """
INSERT INTO songs_in_session (
    sessionId, 
    itemInSession, 
    artist, 
    title, 
    length
) VALUES (%s, %s, %s, %s, %s)
"""

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        # unpack line and execute query
        artist, firstName, gender, itemInSession, \
        lastName, length, level, location, sessionId, song, userId = line
        session.execute(
            query, 
            (int(sessionId), int(itemInSession), artist, song, float(length))
        )

#### Validate

In [10]:
# execute test query
rows = session.execute(
        """
        SELECT artist, title, length 
        FROM songs_in_session 
        WHERE sessionId = 338 AND itemInSession = 4
        """
)

# display results
for row in rows:
    print(row.artist, row.title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


___

### Query 2
**Request**: Select only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The following query achieves the desired results:

```sql
SELECT artist, song, firstName, lastName, itemInSession 
FROM songs_by_user 
WHERE userId = 10 AND sessionId = 182
```

* The table name is be `songs_by_user`.
* Table attributes are: `userId`, `sessionId`, `artist`, `song`, `firstName`, `lastName`, and `itemInSession`.
* The primary key will consist of a composite partition key: `userId`, `sessionId`. This will ensure that `userId` and `sessionId` data will be part of the same node.
* Results are ordered by `itemInSession`.

#### Create Table

In [11]:
# create table for query 2
query = """
CREATE TABLE IF NOT EXISTS songs_by_user (
    userId INT, 
    sessionId INT,
    itemInSession INT,
    artist TEXT, 
    song TEXT, 
    firstName TEXT, 
    lastName TEXT,  
    PRIMARY KEY ((userId, sessionId), itemInSession)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)        

#### Insert Data

In [12]:
# insert data
file = 'event_datafile_new.csv'
query = """
INSERT INTO songs_by_user (
    userId, 
    sessionId,
    itemInSession,
    artist, 
    song, 
    firstName, 
    lastName
) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        # unpack line and execute query
        artist, firstName, gender, itemInSession, \
        lastName, length, level, location, sessionId, song, userId = line
        session.execute(
            query, 
            (
                int(userId), 
                int(sessionId),
                int(itemInSession),
                artist, 
                song, 
                firstName, 
                lastName
            )
                       )

#### Validate

In [13]:
# validate
rows = session.execute(
        """
        SELECT artist, song, firstName, lastName, itemInSession 
        FROM songs_by_user 
        WHERE userId = 10 AND sessionId = 182
        """
)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname, row.iteminsession)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


___

### Query 3
**Request**: Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The following query achieves the desired results:

```sql
SELECT firstName, lastName 
FROM app_history 
WHERE song = 'All Hands Against His Own'
```

* The table name is `app_history`.
* Table attributes are: `song`, `firstName`, `lastName` and `userId`.
* The primary key consists of partition key `song`, and clustering key `userId`.


#### Create Table

In [14]:
# create table
query = """
CREATE TABLE IF NOT EXISTS app_history (
    song text,
    userId int,
    firstName text, 
    lastName text, 
    PRIMARY KEY (song, userId)
)
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data

In [16]:
# insert data
file = 'event_datafile_new.csv'
query = """
INSERT INTO app_history (
    song,
    userId,
    firstName, 
    lastName 
    
) VALUES (%s, %s, %s, %s)
"""

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # unpack line and execute query
        artist, firstName, gender, itemInSession, \
        lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, 
                        (song, int(userId), firstName, lastName )
                       )

#### Validate Results

In [17]:
# validate
rows = session.execute(
        """
        SELECT firstName, lastName 
        FROM app_history 
        WHERE song = 'All Hands Against His Own'
        """
)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


___

### Drop the tables before closing out the sessions

In [18]:
# query 1
session.execute('DROP TABLE IF EXISTS songs_in_session')

# query 2
session.execute('DROP TABLE IF EXISTS songs_by_user')

# query 3
session.execute('DROP TABLE app_history')

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()